# NOTEBOOK 1: INGEST HOUSING DATA INTO SNOWFLAKE

#### Download the housing dataset

In [1]:
# We will load some data from internet which we will then load into Snowflake 
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

In [2]:
# Import Snowpark libraries 
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import *

#Snowflake connection info is saved in config.py
# Please change config.py to point to your Snowflake instance before running this step
# Use user which has account admin role or equivalent to create DB, Schema, warehouse etc.

from config import snowflake_conn_prop

import pandas as pd 

def load_housing_data ( housing_path = HOUSING_PATH ): 
    csv_path = os.path.join ( housing_path , "housing.csv" ) 
    return pd.read_csv ( csv_path ) 

housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


Let's configure our Snowpark Session and initialize the database, warehouse, and schema that we will use for the remainder of the quickstart.

In [3]:
from snowflake.snowpark import version
print(version.VERSION)

session = Session.builder.configs(snowflake_conn_prop).create()
session.use_role("ACCOUNTADMIN")
session.sql("create database if not exists  {}".format(snowflake_conn_prop['database'])).collect()
session.use_database(format(snowflake_conn_prop['database']))
session.sql("create schema if not exists {}".format(snowflake_conn_prop['schema'])).collect()
session.use_schema(format(snowflake_conn_prop['schema']))
session.sql("create or replace warehouse {} with \
                WAREHOUSE_SIZE = XSMALL \
                AUTO_SUSPEND = 120 \
                AUTO_RESUME = TRUE".format(snowflake_conn_prop['warehouse'])).collect()
session.use_warehouse(format(snowflake_conn_prop['warehouse']))
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

(1, 1, 0)
[Row(CURRENT_WAREHOUSE()='LAB_SCIKIT_WH', CURRENT_DATABASE()='SCIKIT_LEARN', CURRENT_SCHEMA()='PUBLIC')]


### Write the data into a Snowflake table named `HOUSING_DATA`

Use the **write_pandas** method to create a table with the data in the Pandas dataframe, housing. The table will be automatically created, if it exists it will be dropped and recreated (auto_create_table and overwrite controls this)

In [4]:
snowpark_df = session.write_pandas(housing, "HOUSING_DATA", quote_identifiers=False, auto_create_table=True, overwrite=True)

Check that we have data in the table

In [5]:
snowpark_df.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"LONGITUDE"  |"LATITUDE"  |"HOUSING_MEDIAN_AGE"  |"TOTAL_ROOMS"  |"TOTAL_BEDROOMS"  |"POPULATION"  |"HOUSEHOLDS"  |"MEDIAN_INCOME"  |"MEDIAN_HOUSE_VALUE"  |"OCEAN_PROXIMITY"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|-122.23      |37.88       |41.0                  |880.0          |129.0             |322.0         |126.0         |8.3252           |452600.0              |NEAR BAY           |
|-122.22      |37.86       |21.0                  |7099.0         |1106.0            |2401.0        |1138.0        |8.3014           |358500.0              |NEAR BAY           |
|-122.24      |37.85       |52.0                  |1467.0         |190.0             |496.0         |177.0    

In [6]:
session.close()